In [18]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [19]:
openai = OpenAI()

In [20]:
###NOTE: Below does not work on JS activated website, that will work only with selenium :)

In [21]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [22]:
##Testing Beautifulsoup on my website lol
ps = Website("https://prith27.github.io/")
print(ps.title)
print(ps.text)

Prithvi Seshadri
Prithvi Seshadri
Data Scientist
Show Contacts
Email
prithvi.seshadri01@gmail.com
Phone
+91 9940654174
Birthday
Sept 27, 2001
Location
Chennai, Tamil Nadu, India
About
Resume
Blog
Contact
About me
Hey there! I'm a Data Scientist at Shell, born in the BBQ haven of Houston, Texas, but currently rocking it in Chennai, India. By day, I wrangle data and deploy cutting-edge ML & DL algorithms to solve Shell’s asset-based mysteries with end-to-end CI/CD pipelines, saving the company millions.
When I’m not crunching numbers, you'll find me on the football field. I've played Division 1 football and represented my school in national competitions, winning numerous local tournaments along the way.
On the nerdy side, I love diving into data science research and have proudly published three papers in the field. Footballer by day, data scientist by... well, also by day. Welcome to my world!
What I do
Data Science
Leveraging data to uncover patterns and developing advanced machine lear

In [23]:
##Creating user and system prompts
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [24]:
##Func for roles to model 
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [25]:
##TEsting role function
messages_for(ps)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Prithvi Seshadri\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nPrithvi Seshadri\nData Scientist\nShow Contacts\nEmail\nprithvi.seshadri01@gmail.com\nPhone\n+91 9940654174\nBirthday\nSept 27, 2001\nLocation\nChennai, Tamil Nadu, India\nAbout\nResume\nBlog\nContact\nAbout me\nHey there! I'm a Data Scientist at Shell, born in the BBQ haven of Houston, Texas, but currently rocking it in Chennai, India. By day, I wrangle data and deploy cutting-edge ML & DL algorithms to solve Shell’s asset-based mysteries with end-to-end CI/CD pipelines, saving the company millions.\nWhen I’m not crunching numbers, you'll 

In [26]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [27]:
summarize("https://prith27.github.io/")

'# Summary of Prithvi Seshadri\'s Website\n\nPrithvi Seshadri is a Data Scientist at Shell, currently based in Chennai, India. Originally from Houston, Texas, he specializes in machine learning and deep learning, focusing on deploying CI/CD pipelines to solve complex problems and save significant costs for his company. Besides his professional work, Prithvi is an avid football player and supporter of Manchester United, with a background in competitive sports.\n\n## Professional Overview\n- **Current Position**: Data Scientist at Shell (Aug 2023 - Present)\n  - Developed CI/CD pipelines on Azure, focusing on predictive maintenance models leading to savings of approximately $9 million.\n- **Previous Experience**: Data Analyst Intern at Acies Global (May 2022 - Jun 2022)\n\n## Education\n- **BTech in Computer Science**: Vellore Institute of Technology (2019 - 2023), specializing in AI and ML.\n- **High School**: DAV Public School, where he ranked third in his computer science department.\

In [28]:
##Better display
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [29]:
display_summary("https://prith27.github.io/")

# Prithvi Seshadri

**Profession**: Data Scientist  
**Current Employer**: Shell  
**Location**: Chennai, Tamil Nadu, India  
**Email**: [prithvi.seshadri01@gmail.com](mailto:prithvi.seshadri01@gmail.com)  
**Phone**: +91 9940654174  
**Birthday**: September 27, 2001  

## About Me
Prithvi Seshadri is a data scientist with a strong background in machine learning (ML) and deep learning (DL). Originally from Houston, Texas, he currently works at Shell, where he develops ML and DL algorithms to improve operational efficiencies, generating substantial cost savings. Outside of his professional life, Prithvi is also an avid football player and enthusiast, previously competing at a Division 1 level.

## Education
- **BTech in Computer Science**  
  **Institution**: Vellore Institute of Technology  
  **Years**: 2019 — 2023  
  **CGPA**: 9.16/10  
- **CBSE AISSCE**  
  **Institution**: DAV Public School  
  **Years**: 2017 — 2019  
  **Score**: 96.2%

## Experience
- **Data Scientist @ Shell** (Aug 2023 — Present)  
  Developed automated CI/CD pipelines and designed predictive maintenance models that resulted in significant savings for the company.
  
- **Data Analyst Intern @ Acies Global** (May 2022 — Jun 2022)  
  Analyzed market trends and collaborated with clients to enhance sales data accuracy.

## Skills
- Machine Learning and Deep Learning: 90%
- MLOps: 90%
- Data Modeling and Analysis: 90%
- Artificial Intelligence: 85%

## Publications
1. **Hand Detection and Morse Code Translation for Alternative Communication** - IEEE International Conference on System, Computation, Automation and Networking (ICSCAN), Nov 18, 2023
2. **Detection of Phishing URLs Using Machine Learning and Deep Learning Models** - Evolution and Applications of Quantum Computing, Jun 2, 2022
3. **A Beginner’s Guide to Training a YOLOv5 Object Detection Model** - Medium.com, Feb 24, 2023
4. **Building Accurate Object Detection Models with RetinaNet** - Medium.com, Feb 20, 2023
5. **Web Based COVID Detection System using Deep Learning** - Journal of Physics: Conference Series, Nov 24, 2021

## Conclusion
Prithvi Seshadri blends a passion for football with a successful career in data science, showcasing his skills through impactful projects and significant academic achievement.